In [ ]:
!pip install nltk

Анна Запорощенко. Запускается только в коллабе, на всякий случай вот ссылка: https://colab.research.google.com/drive/1heNI53qZhVIjnaDU7Tjm_uGtppAZd7T6?usp=sharing

In [ ]:
!pip install spacy

In [ ]:
!pip  install pyLDAvis

In [ ]:
import nltk; nltk.download('stopwords')


!python3 -m spacy download en

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')

In [ ]:
# Convert to list
data = df.content.values.tolist()


#YOURCODE

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

Mallet

In [ ]:

!pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.3)


In [ ]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)


In [ ]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

--2020-10-19 08:23:59--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip.1’

mallet-2.0.8.zip.1  100%[===================>]  15.43M  15.3MB/s    in 1.0s    

2020-10-19 08:24:01 (15.3 MB/s) - ‘mallet-2.0.8.zip.1’ saved [16184794/16184794]

Archive:  mallet-2.0.8.zip
replace mallet-2.0.8/bin/classifier2info? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
mallet_path = '/content/mallet-2.0.8/bin/mallet' # you should NOT need to change this 

Функция, которая подбирает оптимальное количество топиков
Перебираем диапазон топиков от 18 до 28 (очень долго ждать, поэтому так), на каждом обучаем модель. Для каждой модели считаем метрику и выбираем то количество топиков, где coherence максимальная.

In [ ]:
def best_num_topics(mallet_path, corpus, data_lemmatized, id2word):
  max_coherence = 0
  best_number = 0
  for i in range(18,28):
    ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=i, id2word=id2word)
    coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    print(i, ' ', coherence_ldamallet)
    if coherence_ldamallet > max_coherence:
      max_coherence = coherence_ldamallet
      best_number = i
  return best_number

In [ ]:
best_num_topics(mallet_path, corpus, data_lemmatized, id2word)

18   0.5311770708987563
19   0.5398267468211734
20   0.5354354454953549
21   0.5444309282354114
22   0.5283106777577377
23   0.5237340440852971
24   0.5457875766636415
25   0.5328867196436825
26   0.5294259230562613
27   0.5319354273729553


24

Модель, обученная для лучшего количества топиков (24)

In [ ]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=24, id2word=id2word)

In [ ]:
doc_lda = ldamallet[corpus]

У нас есть выдача модели по данным corpus. Там для каждого списка тем есть вероятность, с которой конкретный текст соответствует этой теме. Можно просто выбрать самое большое значение для каждого текста.

In [ ]:
def wide_topic_creation(doc_lda):
  wide_topics = {}
  for i in range(0,24): #генерируем словарь формата "номер топика":"список" (пока пустой)
    wide_topics[i] = []
  for text in enumerate(doc_lda): #для каждого текста задаем счетчик и перебором вытаскиваем самый вероятный топик
    counter = 0
    topic_number = 100
    for topic in text[1]:
      if topic[1] > counter:
        counter = topic[1]
        topic_number = topic[0]
    wide_topics[topic_number] == wide_topics[topic_number].append(text[0]) #каждый раз пополняем список соответствующего топика в словаре
  return wide_topics

In [ ]:
wide_topics = wide_topic_creation(doc_lda)

Теперь считаем tf_idf и сразу будем выбирать по пять слов с самым высоким tf_idf

In [100]:
def td_idf(texts_id, texts): #функция берет id текстов для одного топика и все лемматизированные тексты
  the_texts = []
  for i in texts_id: #достаем только нужные
    the_texts.append((' '.join(texts[i]))) #векторайзер работает с текстом, а не списком слов, поэтому объединяем
  vectorizer = TfidfVectorizer()
  vectors = vectorizer.fit_transform(the_texts)
  feature_names = vectorizer.get_feature_names() #вокабуляр
  dense = vectors.todense()
  denselist = dense.tolist() #список векторов, равных по длине вокабуляру, собственно, tf_idf
  best_five = [] #инициализируем список для наборов по 5 слов
  for text in denselist:
    zipped = zip(text, feature_names) #склеиваем вектор каждого текста с вокабуляром
    zipped = sorted(zipped, key=lambda x: x[0], reverse=True)[0:5] #сортируем по убыванию, оставляем 5 самых больших tf_idf
    id_tdf, words = zip(*zipped) #разлепляем обратно
    best_five.append(words)
  return best_five

In [ ]:
for topic_number, texts_id in wide_topics.items(): #будем пополнять уже созданный словарь wide_topics
  best_five = td_idf(texts_id, texts)
  id_best_five_dict = {} #инициализируем словарь для каждого топика
  for i in enumerate(texts_id):
    id_best_five_dict[data[i[1]]] = list(best_five[i[0]]) #внутри этого словаря находятся пары "исходный текст": "5 слов с самым высоким ft_idf"
  wide_topics[topic_number] = id_best_five_dict # для каждого топика - по такому словарю

Теперь выведем таблицу

In [ ]:
df = pd.DataFrame(wide_topics)
df.stack().reset_index()

,level_0,level_1,0
0,From: (Bob Shaw) Subject: SUMMARY xon and X11R...,0,"[semiconductor, symptom, folk, hi, user]"
1,From: (Benedikt Rosenau) Subject: Re: Gospel D...,0,"[old, argument, obvious, evidence, know]"
2,From: (Bob Douglas) Subject: Re: Sphere from 4...,0,"[plane, normal, lie, point, find]"
3,From: (Mark Zenier) Subject: Re: MC SBI mixer ...,0,"[circuit, theory, receiver, lot, book]"
4,From: (Ferdinand Oeinck) Subject: detecting do...,0,"[detect, ferdinand, point, calculating_double,..."
...,...,...,...
11309,From: (Melinda . Hsu ) Subject: Re: The arroga...,23,"[spiritual, choice, actual, faith, thing]"
11310,From: Subject: Re: The arrogance of Christians...,23,"[evidence, reason, faith, experience, explain]"
11311,From: (Keith Allan Schneider) Subject: Re: Obj...,23,"[omniscient, goal, specie, actualization, exte..."
11312,From: mathew Subject: Alt.Atheism FAQ: Atheist...,23,"[book, atheism, humanism, address, history]"


Coherence score - это метрика качества, которая показывает, с какой вероятностью каждые два ключевых для топика слова могут встретиться вместе. Если в целом, то метрика показывает, насколько вообще близки/сочетаемы ключевые слова, соответственно, насколько похожи будут друг на друга тексты, которые распознаются в качестве подходящих для данного топика.